In [1]:
def github() -> str:
    """
    This function returns a link to the GitHub repository containing solutions.

    Returns:
        str: The URL to the GitHub repository.
    """
    return "https://github.com/anasfathul/ECON481/blob/main/ECON481_HW6.py"

In [2]:
# DESCRIPTION

In [3]:
# Package load
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.orm import Session

In [4]:
# Connect with sqlalchemy
path = 'https://github.com/lukas-hager/econ-481-jupyterhub/blob/main/auctions.db'
engine = create_engine(f'sqlite:///{path}')


In [5]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.orm import Session

In [125]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.orm import Session
path = '/home/jovyan/econ-481-jupyterhub/auctions.db'

class DataBase:
    def __init__(self, loc: str, db_type: str = "sqlite") -> None:
        """Initialize the class and connect to the database"""
        self.loc = loc
        self.db_type = db_type
        self.engine = create_engine(f'{self.db_type}:///{self.loc}')
    def query(self, q: str) -> pd.DataFrame:
        """Run a query against the database and return a DataFrame"""
        with Session(self.engine) as session:
            df = pd.read_sql(q, session.bind)
        return(df)
    def execute(self, q: str) -> None:
        """Execute statement on the database"""
        with self.engine.connect() as conn:
            conn.execute(q)

auctions = DataBase(path)


In [7]:
inspector = inspect(auctions.engine)
inspector.get_table_names()

['bids', 'items']

In [8]:
print(auctions.query('select * from bids').head())

   index  bidLogId     itemId  itemPrice  bidAmount  \
0     50         0  178348858       9.99       20.0   
1     51         0  178348858      13.00       12.0   
2     52         0  178348858      21.00       23.0   
3     53         0  178348858      24.00       35.0   
4     54         0  178348858      36.00       48.0   

                      bidTime  quantity bidIPAddress adCode serverIP  \
0  2023-09-18 16:11:04.587000         1         None   None     None   
1  2023-09-22 14:22:06.700000         1         None   None     None   
2  2023-09-23 12:35:18.157000         1         None   None     None   
3  2023-09-23 18:23:27.993000         1         None   None     None   
4  2023-09-23 18:37:47.213000         1         None   None     None   

   retracted bidderName highBidderName  isBuyerHighBidder  isLoggedInBuyer  
0          0     a****9         a****9                  0                0  
1          0     S****p         a****9                  0                0  
2    

In [9]:
# Exercise 1
def std() -> str:
    """
    This function return sql query that outputs a table that has two columns: itemId and std, the standard deviation of bids for that item.
    """
    query = """
    SELECT bids.itemID, SQRT(SUM((bidAmount - avgBid) * (bidAmount - avgBid)) / (COUNT(bidAmount) - 1)) AS std
    FROM bids
    JOIN (
        SELECT itemID, AVG(bidAmount) AS avgBid
        FROM bids
        GROUP BY itemID
        having COUNT(*) > 1
    ) sub ON bids.itemID = sub.itemID
    GROUP BY bids.itemID
    """
    return query

auctions.query(std())

,itemId,std
0,172998011,0.000000
1,174445924,0.577350
2,174551689,1.830034
3,174767945,8.607399
4,174871788,3.109126
...,...,...
82,182571641,7.767700
83,182660924,4.364634
84,182745253,2.884098
85,182883873,0.707107


In [58]:
# Exercise 2
def bidder_spend_frac() -> str:
    """
    This function return sql query that outputs a table that has four columns:
    bidderName: the name of the bidder
    total_spend: the amount the bidder spent (that is, the sum of their winning bids)
    total_bids: the amount the bidder bid, regardless of the outcome.
    NB: bidders may submit multiple bids for an item – if this is the case only count their highest bid for an item for this calculation.
    spend_frac: total_spend/total_bids
    """
    query = """
    WITH Winner AS (
        SELECT bidderName, SUM(sub.max_bid) as total_spend
        FROM (
            SELECT itemid, bidderName, MAX(bidAmount) as max_bid
            FROM bids
            GROUP BY itemid
        ) AS sub
        GROUP BY bidderName
    ), 
    HighestBid AS (
        SELECT s.bidderName, SUM(s.max_bid) as total_bids
        FROM (
            SELECT itemid, bidderName, MAX(bidAmount) as max_bid
            FROM bids
            GROUP BY itemid, bidderName
        ) as s
        GROUP BY bidderName
    )
    SELECT h.bidderName,
        IFNULL(w.total_spend, 0) as total_spend,
        IFNULL(h.total_bids, 0) as total_bids,
        IFNULL((w.total_spend / h.total_bids), 0) as spend_frac
    FROM HighestBid h
    LEFT JOIN Winner w ON h.bidderName = w.bidderName
    """
    return query

auctions.query(bidder_spend_frac())

,bidderName,total_spend,total_bids,spend_frac
0,1****3,0.00,28.00,0.000000
1,2****h,0.00,35.00,0.000000
2,4****S,0.00,23.99,0.000000
3,7****1,7.99,23.99,0.333055
4,A****3,0.00,383.00,0.000000
...,...,...,...,...
279,v****p,0.00,9.99,0.000000
280,w****5,46.99,46.99,1.000000
281,w****7,19.99,19.99,1.000000
282,y****2,0.00,13.01,0.000000


In [59]:
# CHECK ANSWER DIFFERENT APPROACH FROM CHATGPT, SAME ANSWER

sq_combined = """
WITH HighestBid AS (
    SELECT bidderName, SUM(max_bid) as total_bids
    FROM (
        SELECT itemid, bidderName, MAX(bidAmount) as max_bid
        FROM bids
        GROUP BY itemid, bidderName
    ) as sub
    GROUP BY bidderName
),
Winner AS (
    SELECT highBidderName as bidderName, SUM(bidAmount) as total_spend
    FROM (
        SELECT itemid, highBidderName, MAX(bidAmount) as bidAmount
        FROM bids
        GROUP BY itemid
    ) as sub
    GROUP BY highBidderName
)
SELECT h.bidderName,
    IFNULL(w.total_spend, 0) as total_spend,
    h.total_bids,
    IFNULL((w.total_spend / h.total_bids), 0) as spend_frac
FROM HighestBid h
LEFT JOIN Winner w ON h.bidderName = w.bidderName

"""

auctions.query(sq_combined)


,bidderName,total_spend,total_bids,spend_frac
0,1****3,0.00,28.00,0.000000
1,2****h,0.00,35.00,0.000000
2,4****S,0.00,23.99,0.000000
3,7****1,7.99,23.99,0.333055
4,A****3,0.00,383.00,0.000000
...,...,...,...,...
279,v****p,0.00,9.99,0.000000
280,w****5,46.99,46.99,1.000000
281,w****7,19.99,19.99,1.000000
282,y****2,0.00,13.01,0.000000


In [113]:
# Exercise 3
def min_increment_freq() -> str:
    """
    This function return a SQL query that outputs a table that has one column (freq) which represents
    the fraction of bids in the database that are exactly the minimum bid increment (items.bidIncrement)
    above the previous high bid.
    """
    query = """
    WITH itemAlter AS (
        SELECT itemId, bidIncrement 
        FROM items 
        WHERE isBuyNowUsed != 1
    ),
    bidsWithIncrement AS (
        SELECT 
            b.*, 
            i.bidIncrement
        FROM 
            bids b
        LEFT JOIN 
            itemAlter i ON b.itemId = i.itemId
    ),
    bidsLagged AS (
        SELECT 
            itemid,
            itemprice,
            bidAmount,
            bidIncrement,
            LAG(bidAmount) OVER (PARTITION BY itemid ORDER BY bidtime) AS lagged_bidAmount,
            (bidAmount - LAG(bidAmount) OVER (PARTITION BY itemid ORDER BY bidtime)) AS increment
        FROM bidsWithIncrement
    )
    SELECT 
        CAST(SUM(CASE WHEN increment = bidIncrement THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS freq
    FROM 
        bidsLagged
    """
    return query

auctions.query(min_increment_freq())

# Notes: IDK how to handle those first bid amount as in what is the increment for the first bid, should that excluded from total bids

,freq
0,0.107078


In [114]:
gg = """
WITH itemAlter AS (
    SELECT itemId, bidIncrement 
    FROM items 
    WHERE isBuyNowUsed != 1
),
bidsWithIncrement AS (
    SELECT 
        b.*, 
        i.bidIncrement
    FROM 
        bids b
    LEFT JOIN 
        itemAlter i ON b.itemId = i.itemId
),
bidsLagged AS (
    SELECT 
        itemid,
        itemprice,
        bidAmount,
        bidIncrement,
        LAG(bidAmount) OVER (PARTITION BY itemid ORDER BY bidtime) AS lagged_bidAmount,
        (bidAmount - LAG(bidAmount) OVER (PARTITION BY itemid ORDER BY bidtime)) AS increment
    FROM bidsWithIncrement
)
SELECT 
    CAST(SUM(CASE WHEN increment = bidIncrement THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS freq
FROM 
    bidsLagged
WHERE 
    lagged_bidAmount IS NOT NULL

"""
auctions.query(gg)

,freq
0,0.155673


In [170]:
# Exercise 4
def win_perc_by_timestamp() -> str:
    """
    This function return a SQL query that outputs a table that has two columns:
    timestamp_bin: Using the same methodology as in the slides to normalize the percentage of time remaining in the auction when a bid is placed,
    normalize the bid timestamp and classify it as one of ten bins: 1 corresponds to 0-.1, 2 corresponds to .1-.2, etc.
    win_perc: the frequency with which a bid placed with this timestamp bin won the auction.
    """
    query = """
    WITH partA AS (
        SELECT 
            i.itemid, 
            i.starttime, 
            i.endtime, 
            julianday(i.endtime) - julianday(i.starttime) AS length
        FROM items AS i
    ),
    partitionB AS (
        SELECT 
            b.itemid, 
            b.bidtime, 
            a.starttime, 
            a.endtime,
            CASE 
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.1 THEN 1
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.2 THEN 2
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.3 THEN 3
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.4 THEN 4
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.5 THEN 5
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.6 THEN 6
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.7 THEN 7
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.8 THEN 8
                WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.9 THEN 9
                ELSE 10
            END AS timestamp_bin,
            MAX(b.bidamount) OVER (PARTITION BY b.itemid) AS max_bid,
            b.bidamount
        FROM bids AS b
        INNER JOIN partA AS a
        ON b.itemid = a.itemid
    ),
    partC AS (
        SELECT 
            timestamp_bin, 
            CASE 
                WHEN bidamount = max_bid THEN 1 
                ELSE 0 
            END AS freq
        FROM partitionB
    )
    SELECT 
        timestamp_bin, 
        SUM(freq) * 1.0 / COUNT(*) AS win_perc
    FROM partC
    GROUP BY timestamp_bin
    """
    
    return query


auctions.query(win_perc_by_timestamp())

,timestamp_bin,win_perc
0,1,0.468852
1,2,0.300000
2,3,0.206897
3,4,0.222222
4,5,0.222222
5,6,0.052632
6,7,0.000000
7,8,0.095238
8,9,0.235294
9,10,0.100000


In [169]:
q = """
WITH partA AS (
    SELECT 
        i.itemid, 
        i.starttime, 
        i.endtime, 
        julianday(i.endtime) - julianday(i.starttime) AS length
    FROM items AS i
),
partitionB AS (
    SELECT 
        b.itemid, 
        b.bidtime, 
        a.starttime, 
        a.endtime,
        CASE 
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.1 THEN 1
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.2 THEN 2
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.3 THEN 3
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.4 THEN 4
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.5 THEN 5
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.6 THEN 6
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.7 THEN 7
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.8 THEN 8
            WHEN (julianday(a.endtime) - julianday(b.bidtime)) / a.length <= 0.9 THEN 9
            ELSE 10
        END AS timestamp_bin,
        MAX(b.bidamount) OVER (PARTITION BY b.itemid) AS max_bid,
        b.bidamount
    FROM bids AS b
    INNER JOIN partA AS a
    ON b.itemid = a.itemid
),
partC AS (
    SELECT 
        timestamp_bin, 
        CASE 
            WHEN bidamount = max_bid THEN 1 
            ELSE 0 
        END AS freq
    FROM partitionB
)
SELECT 
    timestamp_bin, 
    SUM(freq) * 1.0 / COUNT(*) AS win_perc
FROM partC
GROUP BY timestamp_bin
"""
auctions.query(q)

,timestamp_bin,win_perc
0,1,0.468852
1,2,0.300000
2,3,0.206897
3,4,0.222222
4,5,0.222222
5,6,0.052632
6,7,0.000000
7,8,0.095238
8,9,0.235294
9,10,0.100000


In [166]:
q = """
with a as (
    select itemid, starttime, endtime, 
    julianday(endtime) - julianday(starttime) as length
    from items
),
partitionB AS (
    select b.itemid, b.bidtime, a.starttime, a.endtime,
    CASE 
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.1 THEN 1
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.2 THEN 2
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.3 THEN 3
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.4 THEN 4
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.5 THEN 5
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.6 THEN 6
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.7 THEN 7
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.8 THEN 8
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.9 THEN 9
        ELSE 10
    END AS timestamp_bin,
    MAX(bidamount) over (partition by itemid) as max_bid
    from bids as b
    inner join a
    on b.itemid=a.item
)


WITH a AS (
    SELECT itemid, starttime, endtime, 
    julianday(endtime) - julianday(starttime) AS length
    FROM items
),
partitionB AS (
SELECT 
    b.itemid, 
    b.bidtime, 
    a.starttime,
    a.endtime,
    (julianday(endtime)-julianday(bidtime)) / a.length AS time_norm,
    CASE 
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.1 THEN 1
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.2 THEN 2
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.3 THEN 3
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.4 THEN 4
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.5 THEN 5
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.6 THEN 6
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.7 THEN 7
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.8 THEN 8
        WHEN (julianday(endtime) - julianday(bidtime)) / a.length <= 0.9 THEN 9
        ELSE 10
    END AS timestamp_bin
FROM bids AS b
INNER JOIN a ON b.itemid = a.itemid
)
SELECT COUNT(timestamp_bin)
FROM partitionB
GROUP BY timestamp_bin
"""
df = auctions.query(q)
print(df)

   COUNT(timestamp_bin)
0                    30
1                    17
2                    21
3                    16
4                    19
5                    27
6                    27
7                    29
8                    60
9                   305


In [153]:
auctions.query('select * from bids')

,index,bidLogId,itemId,itemPrice,bidAmount,bidTime,quantity,bidIPAddress,adCode,serverIP,retracted,bidderName,highBidderName,isBuyerHighBidder,isLoggedInBuyer
0,50,0,178348858,9.99,20.0,2023-09-18 16:11:04.587000,1,None,None,None,0,a****9,a****9,0,0
1,51,0,178348858,13.00,12.0,2023-09-22 14:22:06.700000,1,None,None,None,0,S****p,a****9,0,0
2,52,0,178348858,21.00,23.0,2023-09-23 12:35:18.157000,1,None,None,None,0,H****2,H****2,0,0
3,53,0,178348858,24.00,35.0,2023-09-23 18:23:27.993000,1,None,None,None,0,b****e,b****e,0,0
4,54,0,178348858,36.00,48.0,2023-09-23 18:37:47.213000,1,None,None,None,0,t****5,t****5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,29510,0,177920252,17.00,47.0,2023-09-18 19:21:47.030000,1,None,None,None,0,K****s,K****s,0,0
547,29511,0,177920252,24.50,22.5,2023-09-18 20:00:33.753000,1,None,None,None,0,C****n,K****s,0,0
548,29512,0,177920252,32.00,30.0,2023-09-18 20:00:46.380000,1,None,None,None,0,C****n,K****s,0,0
549,29513,0,177920252,41.00,39.0,2023-09-18 20:01:01.193000,1,None,None,None,0,C****n,K****s,0,0


In [152]:
auctions.query('select endtime from items')

,endTime
0,2023-10-02 18:14:00.000000
1,2023-09-12 19:46:00.000000
2,2023-10-19 04:04:40.000000
3,2023-09-17 18:34:00.000000
4,2023-10-09 19:59:00.000000
...,...
495,2023-11-04 20:18:00.000000
496,2023-09-30 02:39:55.463000
497,2023-09-10 19:13:00.000000
498,2023-10-30 20:13:00.000000


In [92]:
q = """
SELECT 
    itemid,
    itemprice,
    bidAmount,
    LAG(bidAmount) OVER (PARTITION BY itemid ORDER BY bidtime) AS lagged_bidAmount,
    (bidAmount - LAG(bidAmount) OVER (PARTITION BY itemid ORDER BY bidtime)) AS increment
FROM 
    bids
"""
print(auctions.query(q))

        itemId  itemPrice  bidAmount  lagged_bidAmount  increment
0    165561698       9.91       9.91               NaN        NaN
1    170983900       9.91       9.91               NaN        NaN
2    172998011       9.91       9.91               NaN        NaN
3    172998011       9.91       9.91              9.91        0.0
4    173907435      14.99      14.99               NaN        NaN
..         ...        ...        ...               ...        ...
546  182883873      11.99      11.99             10.99        1.0
547  182894197       8.99       8.99               NaN        NaN
548  182925212       7.99       8.01               NaN        NaN
549  182925212      10.01      10.01              8.01        2.0
550  182925212      10.01      10.01             10.01        0.0

[551 rows x 5 columns]


In [111]:
print(auctions.query("""
SELECT 
    itemid,
    itemprice,
    bidAmount,
    min(bidamount) over (partition by itemid) as min_bid,
    LAG(bidAmount) OVER (PARTITION BY itemid ORDER BY bidtime) AS lagged_bidAmount
FROM 
    bids

"""))

        itemId  itemPrice  bidAmount  min_bid  lagged_bidAmount
0    165561698       9.91       9.91     9.91               NaN
1    170983900       9.91       9.91     9.91               NaN
2    172998011       9.91       9.91     9.91               NaN
3    172998011       9.91       9.91     9.91              9.91
4    173907435      14.99      14.99    14.99               NaN
..         ...        ...        ...      ...               ...
546  182883873      11.99      11.99    10.99             10.99
547  182894197       8.99       8.99     8.99               NaN
548  182925212       7.99       8.01     8.01               NaN
549  182925212      10.01      10.01     8.01              8.01
550  182925212      10.01      10.01     8.01             10.01

[551 rows x 5 columns]
